In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/sample_submission.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/all_data.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_public_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test_private_expanded.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/toxicity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/identity_individual_annotations.csv
/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv


In [2]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud, STOPWORDS
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
import math
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc, mean_squared_error
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
import gensim
import string
import tensorflow as tf
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import CuDNNLSTM
from keras.layers import Dropout
import keras.layers 
import warnings
import keras
warnings.filterwarnings("ignore")


In [3]:
train=pd.read_csv("/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/train.csv")

In [4]:
train.head()

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.0,0.000000,0.00000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.0,0.021277,0.87234,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [5]:
train.columns

Index(['id', 'target', 'comment_text', 'severe_toxicity', 'obscene',
       'identity_attack', 'insult', 'threat', 'asian', 'atheist', 'bisexual',
       'black', 'buddhist', 'christian', 'female', 'heterosexual', 'hindu',
       'homosexual_gay_or_lesbian', 'intellectual_or_learning_disability',
       'jewish', 'latino', 'male', 'muslim', 'other_disability',
       'other_gender', 'other_race_or_ethnicity', 'other_religion',
       'other_sexual_orientation', 'physical_disability',
       'psychiatric_or_mental_illness', 'transgender', 'white', 'created_date',
       'publication_id', 'parent_id', 'article_id', 'rating', 'funny', 'wow',
       'sad', 'likes', 'disagree', 'sexual_explicit',
       'identity_annotator_count', 'toxicity_annotator_count'],
      dtype='object')

In [6]:
X_train=train['comment_text']
Y_train=train['target']

In [7]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
def pre_process(text_string):
    text_string = text_string.lower() # Convert everything to lower case.
    text_string = re.sub('[^A-Za-z0-9]+', ' ', text_string) # Remove special characters and punctuations
    
    x = text_string.split()
    new_text = []
    
    for word in x:
        if word not in stop_words:
            new_text.append(stemmer.stem(word))
            
    text_string = ' '.join(new_text)
    return text_string

In [8]:
X_train=X_train.apply(pre_process)
X_train.head()

0    cool like would want mother read realli great ...
1    thank would make life lot less anxieti induc k...
2              urgent design problem kudo take impress
3                        someth abl instal site releas
4                                 haha guy bunch loser
Name: comment_text, dtype: object

In [9]:
for i in range(Y_train.shape[0]):
    if Y_train[i]>=0.5:
        Y_train[i]=1
    else:
        Y_train[i]=0

In [10]:
class LSTMFeaturization:
    
    def __init__(self):
        self.word_mapping = None
        self.total_words = None
        
    
    # Accepts a list of sentences and builds a vocabulary.
    def build_vocabulary(self, sentences):
        
        vocab = set()
        for x in sentences:
            for word in x.split():
                vocab.add(word)
        
        # Create a dictionary from vocabulary.
        vocab_dict = dict.fromkeys(vocab, 0)
        
        # Calculate count of each word..
        for x in sentences:
            for word in x.split():
                vocab_dict[word]+=1
                
        return vocab_dict
    
    
    
    # Accepts a dictionary (vocabulary) and gets the word number in dictionary format
    def get_mapping(self, vocab_dict):
                
        # Get the number of each word into the corpus.
        k = []
        v = []
        for keys,val in vocab_dict.items():
            k.append(keys)
            v.append(val)
            
        kv = np.vstack((k,v)).T
        df = pd.DataFrame(columns=["Word","Count"], data=kv)
        df['Count'] = df['Count'].astype('int')
        
        # Sort the dataframe to get the largest count at first place
        df.sort_values(by=['Count'], ascending=False, inplace=True)
        
        # Give numbering to the most frequent word as 1 then next as 2 and so on.
        df.reset_index(inplace=True)
        df['mapping'] = df.index + 1
        
        df.drop(columns=['index'], inplace=True)
        df.drop(columns=['Count'], inplace=True)
        
        # Convert to dictionary for easier processing.
        dictionary = dict(zip(df['Word'], df['mapping']))
        
        return dictionary
    
    
    # Accepts a list of sentences and generates vocabulary and word mappings.
    def fit(self, sentences):
        v = self.build_vocabulary(sentences)
        self.word_mapping = self.get_mapping(v)
        self.total_words = len(self.word_mapping)
        
    # Converts the sentences to number mappings.
    def transform(self, sentences, pad_length = 350):
        
        whole = list() # Stores mapping for all sentences
        for x in sentences: # for each sentence in list of sentences.
            
            part = list()
            for word in x.split(): # for each word
                if word in self.word_mapping:
                    part.append(self.word_mapping[word]) # Append mapped number.
            whole.append(part) # Append sentence.
        
        # Append additional values to make lengths equal.
        whole = keras.utils.pad_sequences(np.array(whole), maxlen=pad_length)
        
        return whole

In [11]:
lstmfeat = LSTMFeaturization()
lstmfeat.fit(X_train)


In [12]:
lstm_train = lstmfeat.transform(X_train)


In [13]:
print(lstm_train.shape)


(1804874, 350)


In [14]:
from sklearn.model_selection import train_test_split
x_train,x_val,y_train, y_val = train_test_split(lstm_train, Y_train, test_size=0.30, shuffle=True)

In [15]:
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath='/kaggle/working/',
    
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stop=tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy",
    patience=8,
    verbose=1,
    mode="max",
    
)

In [16]:
embedding_vecor_length = 100
total_words = lstmfeat.total_words
model = Sequential()
model.add(keras.layers.Embedding(total_words ,embedding_vecor_length, input_length=350))
model.add(keras.layers.LSTM(128, return_sequences=True))
model.add(keras.layers.LSTM(128))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 350, 100)          23556100  
                                                                 
 lstm (LSTM)                 (None, 350, 128)          117248    
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 23,805,061
Trainable params: 23,805,061
Non-trainable params: 0
_________________________________________________________________


In [18]:
history=model.fit(
    x=x_train,
    y=y_train,
    batch_size=1024,
    epochs=20,
    verbose=1,
    callbacks=[model_checkpoint_callback,early_stop],
    validation_data=[x_val,y_val],
    )

Epoch 1/20
1234/1234 [==============================] - 348s 276ms/step - loss: 0.1530 - accuracy: 0.9434 - val_loss: 0.1315 - val_accuracy: 0.9476
Epoch 2/20
1234/1234 [==============================] - 326s 265ms/step - loss: 0.1250 - accuracy: 0.9497 - val_loss: 0.1317 - val_accuracy: 0.9477
Epoch 3/20
1234/1234 [==============================] - 321s 260ms/step - loss: 0.1149 - accuracy: 0.9528 - val_loss: 0.1351 - val_accuracy: 0.9470
Epoch 4/20
1234/1234 [==============================] - 320s 259ms/step - loss: 0.1032 - accuracy: 0.9575 - val_loss: 0.1465 - val_accuracy: 0.9456
Epoch 5/20
1234/1234 [==============================] - 312s 253ms/step - loss: 0.0909 - accuracy: 0.9624 - val_loss: 0.1623 - val_accuracy: 0.9426
Epoch 6/20
1234/1234 [==============================] - 320s 259ms/step - loss: 0.0786 - accuracy: 0.9676 - val_loss: 0.1736 - val_accuracy: 0.9404
Epoch 7/20
1234/1234 [==============================] - 311s 252ms/step - loss: 0.0672 - accuracy: 0.9726 - val_

In [19]:
test=pd.read_csv('/kaggle/input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')
X_test=test['comment_text']
X_test=X_test.apply(pre_process)
lstm_test = lstmfeat.transform(X_test)


In [20]:
precdtion=model.predict(lstm_test)

3042/3042 [==============================] - 51s 17ms/step


In [21]:
submit=pd.DataFrame()
submit['id']=test['id']
submit['prediction']=precdtion
submit.to_csv("submission.csv",index=False)